<a href="https://colab.research.google.com/github/vivek11416/dataScienceProjects/blob/main/dog_breed_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# download kaggle.json from kaggle account settings page
from google.colab import files
files.upload()

In [5]:
#install kaggle API client
!pip install -q kaggle

In [6]:
#the kaggle API client expects this file to be in kaggle folder
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

#update permission to avoig kaggle tool warning
!chmod 600 ~/.kaggle/kaggle.json

In [7]:
#change current working directory
!mkdir dog_dataset
%cd dog_dataset

/content/dog_dataset


In [8]:
#getting dataset
!kaggle datasets list -s dogbreedidfromcomp

ref                                title                    size  lastUpdated          downloadCount  
---------------------------------  ----------------------  -----  -------------------  -------------  
catherinehorng/dogbreedidfromcomp  dog-breed-id-from-comp  691MB  2020-06-26 03:09:05            269  


In [9]:
#downloading
!kaggle datasets download catherinehorng/dogbreedidfromcomp

 98% 678M/691M [00:21<00:00, 83.7MB/s]
100% 691M/691M [00:21<00:00, 33.9MB/s]
